In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [30]:
df = pd.read_pickle(r'Final-v1.pickle')

In [31]:
df["postal_code"] = df["postal_code"].astype('int')
df["postal_code"] = df["postal_code"].astype('category')

In [32]:
df = df.rename(columns={"Nearest_Bahnhof_km": "Bahnhof","Nearest_Hospital_km":"Hospital",
                        "Nearest_Community_Centre_km": "Community_Centre","Nearest_Cafe_km":"Restaurant",
                        "Nearest_Biergarten_km": "Biergarten","Nearest_Kino_km":"Kino",
                        "Nearest_Parking_km": "Parking","Nearest_university_km":"university",})

In [51]:
df.to_pickle("end_result-v2.pickle")

In [52]:
df.to_csv("end_result-v2.csv")

In [14]:
from patsy import dmatrices

In [36]:
%%capture
#gather features
#features = "+".join(data.columns - ["Charging_time_in_hours_log_New"])

# get y and X dataframes based on this regression:
y, X = dmatrices('Charging_Time_in_hours ~ Count_Charging+Temperature+Precipitation+pd+Bahnhof+Restaurant+Biergarten+Community_Centre+Hospital+Kino+Parking+university' , df, return_type='dataframe')

In [37]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [38]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

In [39]:
vif["features"] = X.columns

In [40]:
vif.round(5)

,VIF Factor,features
0,7.22251,Intercept
1,1.10318,Count_Charging
2,1.00138,Temperature
3,1.00635,Precipitation
4,1.07193,pd
5,1.07167,Bahnhof
6,1.09989,Restaurant
7,1.31501,Biergarten
8,1.05388,Community_Centre
9,1.08340,Hospital


In [41]:
trial_df = df.copy()

In [42]:
trial_df['trial_bhna'] = round(trial_df['Bahnhof'],2)
trial_df['traiL_Hospital'] = round(trial_df['Hospital'],2)
trial_df['traiL_Cc'] = round(trial_df['Community_Centre'],2)

trial_df['trial_Restaurant'] = round(trial_df['Restaurant'],2)
trial_df['traiL_Biergarten'] = round(trial_df['Biergarten'],2)
trial_df['traiL_Kino'] = round(trial_df['Kino'],2)

trial_df['traiL_Parking'] = round(trial_df['Parking'],2)
trial_df['traiL_university'] = round(trial_df['university'],2)

##  Only two parking station in the range of 2 km where there are Banhof,Restaurant,Parking,Hospital,Kino,Community_center,Biergarten 

In [43]:
df1 =    trial_df[(trial_df['trial_bhna'] >= 0.1) & (trial_df['trial_bhna'] < 2) & (trial_df['traiL_Hospital'] >= 0.1) & (trial_df['traiL_Hospital'] < 2)
        & (trial_df['traiL_Cc'] >= 0.1) & (trial_df['traiL_Cc'] < 2) & (trial_df['trial_Restaurant'] >= 0.1) & (trial_df['trial_Restaurant'] < 2)
        & (trial_df['traiL_Biergarten'] >= 0.1) & (trial_df['traiL_Biergarten'] < 2) & (trial_df['traiL_Kino'] >= 0.1) & (trial_df['traiL_Kino'] < 2)
        & (trial_df['traiL_Parking'] >= 0.1) & (trial_df['traiL_Parking'] < 2)]

In [44]:
df1

,park_id,clc_code,area_code,evse_id,datetime_start,datetime_stop,city,latitude,longitude,postal_code,...,Parking,university,trial_bhna,traiL_Hospital,traiL_Cc,trial_Restaurant,traiL_Biergarten,traiL_Kino,traiL_Parking,traiL_university
707210,sl_23895958,Suburban,091620000000,sl_23898218,2019-07-02 13:46:33,2019-07-02 13:46:39,Unna,51.5292,7.6852,42424,...,0.775138,16.292169,1.18,0.80,0.63,0.57,0.40,0.58,0.78,16.29
707211,sl_23895958,Suburban,091620000000,sl_23898218,2019-07-13 10:47:03,2019-07-13 18:17:46,Unna,51.5292,7.6852,42424,...,0.775138,16.292169,1.18,0.80,0.63,0.57,0.40,0.58,0.78,16.29
707212,sl_23895958,Suburban,091620000000,sl_23898218,2019-07-25 23:08:56,2019-07-25 23:09:49,Unna,51.5292,7.6852,42424,...,0.775138,16.292169,1.18,0.80,0.63,0.57,0.40,0.58,0.78,16.29
707213,sl_23895958,Suburban,091620000000,sl_23898218,2019-08-07 15:05:36,2019-08-07 15:05:40,Unna,51.5292,7.6852,42424,...,0.775138,16.292169,1.18,0.80,0.63,0.57,0.40,0.58,0.78,16.29
707214,sl_23895958,Suburban,091620000000,sl_23898218,2019-08-09 16:34:40,2019-08-09 18:51:30,Unna,51.5292,7.6852,42424,...,0.775138,16.292169,1.18,0.80,0.63,0.57,0.40,0.58,0.78,16.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739404,sl_512573,Urban,091620000000,sl_23896687,2021-08-11 15:28:25,2021-08-11 22:54:45,Schweinfurt,50.0472,10.2284,97421,...,0.252831,1000.000000,1.30,0.46,0.29,0.21,0.32,0.40,0.25,1000.00
739405,sl_512573,Urban,091620000000,sl_23896687,2021-08-11 19:56:15,2021-08-11 01:08:10,Schweinfurt,50.0472,10.2284,97421,...,0.252831,1000.000000,1.30,0.46,0.29,0.21,0.32,0.40,0.25,1000.00
739406,sl_512573,Urban,091620000000,sl_23896687,2021-08-11 19:56:15,2021-08-11 15:27:49,Schweinfurt,50.0472,10.2284,97421,...,0.252831,1000.000000,1.30,0.46,0.29,0.21,0.32,0.40,0.25,1000.00
739407,sl_512573,Urban,091620000000,sl_23896687,2021-08-11 19:56:15,2021-08-11 15:28:30,Schweinfurt,50.0472,10.2284,97421,...,0.252831,1000.000000,1.30,0.46,0.29,0.21,0.32,0.40,0.25,1000.00


In [45]:
len(df1.park_id.value_counts())

2

## If we take university , Community centre,Restaurant,Kino,Biergarten,Parking,Hospital

In [46]:
df2 =    trial_df[(trial_df['traiL_university'] >= 0.1) & (trial_df['traiL_university'] < 2) & (trial_df['traiL_Hospital'] >= 0.1) & (trial_df['traiL_Hospital'] < 2)
        & (trial_df['traiL_Cc'] >= 0.1) & (trial_df['traiL_Cc'] < 2) & (trial_df['trial_Restaurant'] >= 0.1) & (trial_df['trial_Restaurant'] < 2)
        & (trial_df['traiL_Biergarten'] >= 0.1) & (trial_df['traiL_Biergarten'] < 2) & (trial_df['traiL_Kino'] >= 0.1) & (trial_df['traiL_Kino'] < 2)
        & (trial_df['traiL_Parking'] >= 0.1) & (trial_df['traiL_Parking'] < 2)]

In [47]:
len(df2.park_id.value_counts())

40

## If we take Hospital Community centre,Restaurant,Kino,Biergarten,Parking

In [48]:
df3 =    trial_df[(trial_df['traiL_Hospital'] >= 0.1) & (trial_df['traiL_Hospital'] < 2)
        & (trial_df['traiL_Cc'] >= 0.1) & (trial_df['traiL_Cc'] < 2) & (trial_df['trial_Restaurant'] >= 0.1) & (trial_df['trial_Restaurant'] < 2)
        & (trial_df['traiL_Biergarten'] >= 0.1) & (trial_df['traiL_Biergarten'] < 2) & (trial_df['traiL_Kino'] >= 0.1) & (trial_df['traiL_Kino'] < 2)
        & (trial_df['traiL_Parking'] >= 0.1) & (trial_df['traiL_Parking'] < 2)]

In [49]:
len(df3.park_id.unique())

63